### NLP ön işleme pratiği
    Kullanılan teknikler :
- postag (sıfat/zamir gibi kelime türlerini bulma)
- stopwords (tek başına anlamsız kelimeleri kaldırmak için)
- counter (kelime sıklığı için)
- Lemmatization / Stemming (Kök bulmak için)

## Twitter iklim krizi verileri ile nlp ön işleme

In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from timeit import default_timer as timer
from datetime import timedelta
import string

In [3]:
df = pd.read_excel("iklim_krizi.xlsx")
df.head()

,tarih,id,icerik,ad,konum,begeni,rt,takipci
0,2022-12-31 22:59:36,1609323416154243072,@Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA \n\nN...,bazenzorgeliyo,NaN,1,0,145
1,2022-12-31 21:12:22,1609296428232510976,@wolfintoaction Kurak geçen bir yıl var ortada...,yesiturk,NaN,1,0,376
2,2022-12-31 17:09:59,1609235429089328896,2021 ; Doğanın bize verdiklerinin yanında bizi...,guvenada,NaN,1,0,3263
3,2022-12-31 17:01:48,1609233370898202880,Erzurum’un yeni yıla karsız girecek olması… ko...,kizginbirkiz,NaN,7,0,30
4,2022-12-31 15:08:54,1609204959177854976,[Ankara'nın iklim gündemi-9] HDP: İklim krizi ...,yesillersol_izm,izmir,1,1,1854


In [4]:
df.isna().sum()

tarih          0
id             0
icerik         0
ad             0
konum      13342
begeni         0
rt             0
takipci        0
dtype: int64

In [5]:
df.drop(columns="konum",axis=1,inplace = True)
df.head()

,tarih,id,icerik,ad,begeni,rt,takipci
0,2022-12-31 22:59:36,1609323416154243072,@Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA \n\nN...,bazenzorgeliyo,1,0,145
1,2022-12-31 21:12:22,1609296428232510976,@wolfintoaction Kurak geçen bir yıl var ortada...,yesiturk,1,0,376
2,2022-12-31 17:09:59,1609235429089328896,2021 ; Doğanın bize verdiklerinin yanında bizi...,guvenada,1,0,3263
3,2022-12-31 17:01:48,1609233370898202880,Erzurum’un yeni yıla karsız girecek olması… ko...,kizginbirkiz,7,0,30
4,2022-12-31 15:08:54,1609204959177854976,[Ankara'nın iklim gündemi-9] HDP: İklim krizi ...,yesillersol_izm,1,1,1854


In [6]:
correlation_matrix = df[['begeni','rt', 'takipci']].corr()
print(correlation_matrix)

           begeni        rt   takipci
begeni   1.000000  0.696226  0.272222
rt       0.696226  1.000000  0.180406
takipci  0.272222  0.180406  1.000000


### NLP kütüphaneleri

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer


import warnings
warnings.filterwarnings("ignore")  # Uyarı mesajlarını gizleyelim.

In [9]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


# POS tag'leri WordNet formatına dönüştürmek için fonksiyon
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Varsayılan olarak isim alır

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\menes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\menes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\menes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\menes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
metin = df["icerik"].tolist()
metin

['@Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA \n\nNeyse komünist olmak da güzel şey eşitlik falan savunuyorsun iklim krizi olmuyor',
 '@wolfintoaction Kurak geçen bir yıl var ortada kuraklık iyice baş gösterip gıda ve iklim krizi olursa beton ve diğer menkul yahut gayrimenkullerin yenilip içilmediği için gıdadan daha fazla önem teşkil etmeyecek gün gelir çerez parası da olur makyajlı beton evler',
 '2021 ; Doğanın bize verdiklerinin yanında bizim onu nasıl hunharca yok ettiğimizi hatırlattı.. Adına ister iklim krizi deyin, ister başka birşey.. https://t.co/2rtifFHMfY',
 'Erzurum’un yeni yıla karsız girecek olması… korkunç… küresel ısınma… yok olan mevsimler… iklim krizi… her şeyin değişmesi… normal hiçbir şey kalmaması…',
 "[Ankara'nın iklim gündemi-9] HDP: İklim krizi yeni kazanç kapısı olamaz - Yeşil Gazete https://t.co/0NGWYjSFAr",
 'iklim krizi yüzünden, yıl başı bu sene yaza denk gelmiş gibi #Noel',
 "2023'te dünyayı Rusya-Ukrayna krizinin etkileri ve enerji krizi riski bekliyor: 202

In [11]:
metin_list = " ".join(metin)
kelimeler = nltk.word_tokenize(metin_list)
counter = Counter(kelimeler)
counter

Counter({',': 34239,
         '.': 33781,
         '#': 27837,
         'iklim': 27435,
         'krizi': 25876,
         ':': 19467,
         '@': 16313,
         've': 15180,
         'https': 15085,
         'bir': 10018,
         'bu': 5530,
         'için': 5442,
         '’': 5264,
         '!': 5199,
         'iklimkrizi': 5151,
         '?': 5043,
         'de': 4882,
         'ile': 4489,
         'da': 4456,
         'İklim': 3943,
         'var': 3113,
         "''": 3083,
         '...': 2750,
         '``': 2712,
         'gibi': 2643,
         'çok': 2624,
         'ne': 2553,
         'yok': 2515,
         'daha': 2485,
         'küresel': 2415,
         'en': 2133,
         'değil': 2109,
         '..': 2059,
         'Bu': 2012,
         ')': 1986,
         '“': 1891,
         '”': 1844,
         'kadar': 1777,
         'olarak': 1719,
         'her': 1615,
         '(': 1589,
         'diye': 1583,
         ';': 1511,
         'dünya': 1486,
         'olan': 1470,
   

In [12]:
kelimeler = nltk.word_tokenize(metin_list)
print("Tokens : ",kelimeler[:10]) ## Jupyter Notebook sunucusu çok büyük verilerde çıktıyı vermiyordu. Bu yüzden ilk 10 veri gösterildi.

Tokens :  ['@', 'Fapej_in_Tufegi', 'YAAAAAAAAAAAAAAAAAAAAAA', 'Neyse', 'komünist', 'olmak', 'da', 'güzel', 'şey', 'eşitlik']


### Köklendirme 
- Stemming veya lemmatization kullanılır.
- Stemming direkt köke odaklanır
- Lemmatization anlamlı köklere odaklanır (better olan kelimenin kökü good olması gibi)

In [29]:
ps = PorterStemmer()
koklendirme = [ps.stem(kok) for kok in kelimeler]
print("kok sonrasi : ",kelimeler[:200])
print("-"*100)
print("kok sonrasi : ",koklendirme[:200])

kok sonrasi :  ['@', 'Fapej_in_Tufegi', 'YAAAAAAAAAAAAAAAAAAAAAA', 'Neyse', 'komünist', 'olmak', 'da', 'güzel', 'şey', 'eşitlik', 'falan', 'savunuyorsun', 'iklim', 'krizi', 'olmuyor', '@', 'wolfintoaction', 'Kurak', 'geçen', 'bir', 'yıl', 'var', 'ortada', 'kuraklık', 'iyice', 'baş', 'gösterip', 'gıda', 've', 'iklim', 'krizi', 'olursa', 'beton', 've', 'diğer', 'menkul', 'yahut', 'gayrimenkullerin', 'yenilip', 'içilmediği', 'için', 'gıdadan', 'daha', 'fazla', 'önem', 'teşkil', 'etmeyecek', 'gün', 'gelir', 'çerez', 'parası', 'da', 'olur', 'makyajlı', 'beton', 'evler', '2021', ';', 'Doğanın', 'bize', 'verdiklerinin', 'yanında', 'bizim', 'onu', 'nasıl', 'hunharca', 'yok', 'ettiğimizi', 'hatırlattı', '..', 'Adına', 'ister', 'iklim', 'krizi', 'deyin', ',', 'ister', 'başka', 'birşey', '..', 'https', ':', '//t.co/2rtifFHMfY', 'Erzurum', '’', 'un', 'yeni', 'yıla', 'karsız', 'girecek', 'olması…', 'korkunç…', 'küresel', 'ısınma…', 'yok', 'olan', 'mevsimler…', 'iklim', 'krizi…', 'her', 'şeyin', 'de

In [45]:
lemmatizer = WordNetLemmatizer()
lema_kelimeler = [lemmatizer.lemmatize(kelime) for kelime in kelimeler]
print("Lemmatize Edilmiş Kelimeler:", " ".join(lema_kelimeler[:200]))

Lemmatize Edilmiş Kelimeler: @ Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA Neyse komünist olmak da güzel şey eşitlik falan savunuyorsun iklim krizi olmuyor @ wolfintoaction Kurak geçen bir yıl var ortada kuraklık iyice baş gösterip gıda ve iklim krizi olursa beton ve diğer menkul yahut gayrimenkullerin yenilip içilmediği için gıdadan daha fazla önem teşkil etmeyecek gün gelir çerez parası da olur makyajlı beton evler 2021 ; Doğanın bize verdiklerinin yanında bizim onu nasıl hunharca yok ettiğimizi hatırlattı .. Adına ister iklim krizi deyin , ister başka birşey .. http : //t.co/2rtifFHMfY Erzurum ’ un yeni yıla karsız girecek olması… korkunç… küresel ısınma… yok olan mevsimler… iklim krizi… her şeyin değişmesi… normal hiçbir şey kalmaması… [ Ankara'nın iklim gündemi-9 ] HDP : İklim krizi yeni kazanç kapısı olamaz - Yeşil Gazete http : //t.co/0NGWYjSFAr iklim krizi yüzünden , yıl başı bu sene yaza denk gelmiş gibi # Noel 2023'te dünyayı Rusya-Ukrayna krizinin etkileri ve enerji krizi riski 

In [49]:
temiz_metin = re.sub(r'http\S+|www\S+|https\S+', '', metin_list, flags=re.MULTILINE)
temiz_metin = re.sub(r'\W', ' ', temiz_metin)
print(f"Temizlenmiş Metin:", temiz_metin[:200])
print("-"*100)
print(f"Ham metin :",metin_list[:200])


Temizlenmiş Metin:  Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA   Neyse komünist olmak da güzel şey eşitlik falan savunuyorsun iklim krizi olmuyor  wolfintoaction Kurak geçen bir yıl var ortada kuraklık iyice baş gösterip g
----------------------------------------------------------------------------------------------------
Ham metin : @Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA 

Neyse komünist olmak da güzel şey eşitlik falan savunuyorsun iklim krizi olmuyor @wolfintoaction Kurak geçen bir yıl var ortada kuraklık iyice baş gösterip g


In [53]:
stop_kelimeler = set(stopwords.words('turkish'))

filtrelenmis_kelimeler = [kelime for kelime in nltk.word_tokenize(temiz_metin) if kelime.lower() not in stop_kelimeler]
stop_words = [kelime for kelime in nltk.word_tokenize(temiz_metin) if kelime.lower() in stop_kelimeler]
print("Filtrelenmiş Kelimeler:", " ".join(filtrelenmis_kelimeler[:200]))
print("Yakalanmış Stopwords  :", " ".join(stop_words[:200]))

Filtrelenmiş Kelimeler: Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA Neyse komünist olmak güzel eşitlik falan savunuyorsun iklim krizi olmuyor wolfintoaction Kurak geçen bir yıl var ortada kuraklık iyice baş gösterip gıda iklim krizi olursa beton diğer menkul yahut gayrimenkullerin yenilip içilmediği gıdadan fazla önem teşkil etmeyecek gün gelir çerez parası olur makyajlı beton evler 2021 Doğanın bize verdiklerinin yanında bizim onu hunharca yok ettiğimizi hatırlattı Adına ister iklim krizi deyin ister başka Erzurum un yeni yıla karsız girecek olması korkunç küresel ısınma yok olan mevsimler iklim krizi şeyin değişmesi normal hiçbir kalmaması Ankara nın iklim gündemi 9 HDP İklim krizi yeni kazanç kapısı olamaz Yeşil Gazete iklim krizi yüzünden yıl başı sene yaza denk gelmiş Noel 2023 te dünyayı Rusya Ukrayna krizinin etkileri enerji krizi riski bekliyor 2023 te dünyayı Rusya Ukrayna savaşının devam etmesi petrol fiyatlarının artmasıyla meydana gelen enerji krizi gıda güvensizliği ekonomik k

In [55]:
kucuk_harfli_metin = metin_list.lower()

In [57]:
normalizasyon_metin = re.sub(r'[^\w\s]', '', kucuk_harfli_metin)

In [59]:
def preprocessing(series, remove_hashtag=False, remove_mentions=False, remove_links=False, remove_numbers=False,
                  remove_short_text=False, lowercase=False, remove_punctuation=False, remove_stopwords=False,
                  remove_rare_words=False, rare_limit=5):    
    if lowercase:
        print("Küçük harfe çeviriliyor...")
        start = timer()
        series = series.str.lower()
        print(f"Küçük harfe çevirme işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_hashtag:
        print("Hashtagler kaldırılıyor...")
        start = timer()
        series = series.str.replace(r'((#)[^\s]*)\b', '', regex=True)
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_mentions:
        print("Mentionlar kaldırılıyor...")
        start = timer()
        series = series.str.replace(r'((@)[^\s]*)\b', '', regex=True)
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_links:
        print("Linkler kaldırılıyor...")
        start = timer()
        series = series.str.replace(r'\n', '', regex=True)
        series = series.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_numbers:
        print("Sayılar kaldırılıyor...")
        start = timer()
        series = series.str.replace(r'\d+', '', regex=True)
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_short_text:
        print("Kısa metinler kaldırılıyor...")
        start = timer()
        series = series.apply(lambda x: re.sub(r'\b\w{1,2}\b', '', x))
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_punctuation:
        print("Noktalama işaretleri kaldırılıyor...")
        start = timer()
        series = series.str.replace(r"((')[^\s]*)\b", '', regex=True)
        series = series.str.replace(r'[^\w\s]', '', regex=True)
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    if remove_stopwords:
        print("Removing stopwords...")
        start = timer()
        # Stopwordsleri veri setinden kaldırma işlemi
        series = series.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_kelimeler]))
        print(f"Stopwords are removed in {timedelta(seconds=timer() - start)}")

    if remove_rare_words:
        print("Nadir kelimeler kaldırılıyor...")
        start = timer()
        whole_count = pd.Series(" ".join(series).split()).value_counts()
        print(f"Seride toplam {whole_count.count()} kelime var.")
        print(f"%{round(whole_count[whole_count <= rare_limit].count() / whole_count.count() * 100, 2)} oranında kelime, nadir limitinden ({rare_limit}) daha az kez görünüyor.")
        to_remove = whole_count[whole_count <= rare_limit]
        print("Nadir kelimeler kaldırılıyor...")
        series = series.apply(lambda x: " ".join(x for x in x.split() if x not in to_remove))
        print(f"{len(to_remove)} nadir kelime kaldırıldı.")
        print(f"Kaldırma işlemi {timedelta(seconds=timer() - start)} içinde tamamlandı.")

    return series


def tr_en_char_translate(series):
    """
    Tasks
    -----
        Convert Turkish characters to English characters.

    Parameters
    ----------
    series: pandas.Series
        The series to be translated.

    Returns
    -------
    series: pandas.Series
        The translated series.
    """
    series = series.str.replace('ı', 'i')
    series = series.str.replace('ü', 'u')
    series = series.str.replace('ö', 'o')
    series = series.str.replace('ğ', 'g')
    series = series.str.replace('ş', 's')
    series = series.str.replace('ç', 'c')
    return series

In [61]:
df['icerik_lowercased'] = preprocessing(df['icerik'], lowercase=True)
df[['icerik', 'icerik_lowercased']].head(20)

Küçük harfe çeviriliyor...
Küçük harfe çevirme işlemi 0:00:00.056382 içinde tamamlandı.


,icerik,icerik_lowercased
0,@Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA \n\nN...,@fapej_in_tufegi yaaaaaaaaaaaaaaaaaaaaaa \n\nn...
1,@wolfintoaction Kurak geçen bir yıl var ortada...,@wolfintoaction kurak geçen bir yıl var ortada...
2,2021 ; Doğanın bize verdiklerinin yanında bizi...,2021 ; doğanın bize verdiklerinin yanında bizi...
3,Erzurum’un yeni yıla karsız girecek olması… ko...,erzurum’un yeni yıla karsız girecek olması… ko...
4,[Ankara'nın iklim gündemi-9] HDP: İklim krizi ...,[ankara'nın iklim gündemi-9] hdp: i̇klim krizi...
5,"iklim krizi yüzünden, yıl başı bu sene yaza de...","iklim krizi yüzünden, yıl başı bu sene yaza de..."
6,2023'te dünyayı Rusya-Ukrayna krizinin etkiler...,2023'te dünyayı rusya-ukrayna krizinin etkiler...
7,@huseyinkanturk gıda ve ilaç stoklamak mı yoks...,@huseyinkanturk gıda ve ilaç stoklamak mı yoks...
8,"Sanat,spor,iklim krizi,ileri dönüşüm odağımızd...","sanat,spor,iklim krizi,ileri dönüşüm odağımızd..."
9,[Ankara'nın iklim gündemi-9] HDP: İklim krizi ...,[ankara'nın iklim gündemi-9] hdp: i̇klim krizi...


In [63]:
df['icerik_mentions'] = preprocessing(df['icerik_lowercased'], remove_mentions=True)
df['icerik_links'] = preprocessing(df['icerik_mentions'], remove_links=True)
df['icerik_numbers'] = preprocessing(df['icerik_links'], remove_numbers=True)
df['icerik_short_text'] = preprocessing(df['icerik_numbers'], remove_short_text=True)
df['icerik_punctuation'] = preprocessing(df['icerik_short_text'], remove_punctuation=True)
df['icerik_rare_words'] = preprocessing(df['icerik_punctuation'], remove_rare_words=True, rare_limit=1)
df['icerik_stopwords'] = preprocessing(df['icerik_rare_words'], remove_stopwords=True)
df['icerik_tr_en_char'] = tr_en_char_translate(df['icerik_stopwords'])


Mentionlar kaldırılıyor...
Kaldırma işlemi 0:00:00.030039 içinde tamamlandı.
Linkler kaldırılıyor...
Kaldırma işlemi 0:00:00.053703 içinde tamamlandı.
Sayılar kaldırılıyor...
Kaldırma işlemi 0:00:00.091331 içinde tamamlandı.
Kısa metinler kaldırılıyor...
Kaldırma işlemi 0:00:00.165192 içinde tamamlandı.
Noktalama işaretleri kaldırılıyor...
Kaldırma işlemi 0:00:00.114298 içinde tamamlandı.
Nadir kelimeler kaldırılıyor...
Seride toplam 93814 kelime var.
%63.46 oranında kelime, nadir limitinden (1) daha az kez görünüyor.
Nadir kelimeler kaldırılıyor...
59530 nadir kelime kaldırıldı.
Kaldırma işlemi 0:00:01.366426 içinde tamamlandı.
Removing stopwords...
Stopwords are removed in 0:00:00.130652


In [67]:
df[['icerik', 'icerik_tr_en_char']].head(20)

,icerik,icerik_tr_en_char
0,@Fapej_in_Tufegi YAAAAAAAAAAAAAAAAAAAAAA \n\nN...,neyse komunist olmak guzel esitlik falan iklim...
1,@wolfintoaction Kurak geçen bir yıl var ortada...,kurak gecen bir yil var ortada kuraklik iyice ...
2,2021 ; Doğanın bize verdiklerinin yanında bizi...,doganin bize yaninda bizim onu hunharca yok ha...
3,Erzurum’un yeni yıla karsız girecek olması… ko...,erzurum yeni yila karsiz girecek olmasi korkun...
4,[Ankara'nın iklim gündemi-9] HDP: İklim krizi ...,ankara iklim gundemi hdp klim krizi yeni kazan...
5,"iklim krizi yüzünden, yıl başı bu sene yaza de...",iklim krizi yuzunden yil basi sene yaza denk g...
6,2023'te dünyayı Rusya-Ukrayna krizinin etkiler...,dunyayi rusyaukrayna krizinin etkileri enerji ...
7,@huseyinkanturk gıda ve ilaç stoklamak mı yoks...,gida ilac stoklamak yoksa bunlarin almak yarar...
8,"Sanat,spor,iklim krizi,ileri dönüşüm odağımızd...",donusum arttigi gidecek yolumuz gorecek yillar...
9,[Ankara'nın iklim gündemi-9] HDP: İklim krizi ...,ankara iklim gundemi hdp klim krizi yeni kazan...


In [69]:

# Metni temizle ve kelimelere ayır
def preprocess_text(text):
    words = text.split()  # Kelimelere ayır
    return words


def analyze_columns(df, column1, column2):
    # Sütunlardaki metinleri birleştir ve kelimelere ayır
    print("Sütunlardaki metinler birleştiriliyor.")
    words1 = df[column1].apply(preprocess_text).sum()
    words2 = df[column2].apply(preprocess_text).sum()

    # Toplam kelime sayıları
    print("Toplam kelime sayıları hesaplanıyor.")
    total_words1 = len(words1)
    total_words2 = len(words2)

    # Farklı kelime sayıları
    print("Farklı kelime sayıları hesaplanıyor.")
    unique_words1 = len(set(words1))
    unique_words2 = len(set(words2))

    # En çok tekrar eden 10 kelime
    print("En çok tekrar eden 10 kelime hesaplanıyor.")
    common_words1 = Counter(words1).most_common(10)
    common_words2 = Counter(words2).most_common(10)

    # En az tekrar eden 10 kelime
    print("En az tekrar eden 10 kelime hesaplanıyor.")
    least_common_words1 = Counter(words1).most_common()[:-11:-1]
    least_common_words2 = Counter(words2).most_common()[:-11:-1]

    # Sonuçları ekrana bas
    print(f"Sütun 1: {column1}")
    print(f"Toplam kelime sayısı: {total_words1}")
    print(f"Farklı kelime sayısı: {unique_words1}")
    print(f"En çok tekrar eden 10 kelime: {common_words1}")
    print(f"En az tekrar eden 10 kelime: {least_common_words1}")
    print("\n")
    print(f"Sütun 2: {column2}")
    print(f"Toplam kelime sayısı: {total_words2}")
    print(f"Farklı kelime sayısı: {unique_words2}")
    print(f"En çok tekrar eden 10 kelime: {common_words2}")
    print(f"En az tekrar eden 10 kelime: {least_common_words2}")


# DataFrame'deki sütunları analiz et
analyze_columns(df, 'icerik', 'icerik_tr_en_char')


Sütunlardaki metinler birleştiriliyor.
Toplam kelime sayıları hesaplanıyor.
Farklı kelime sayıları hesaplanıyor.
En çok tekrar eden 10 kelime hesaplanıyor.
En az tekrar eden 10 kelime hesaplanıyor.
Sütun 1: icerik
Toplam kelime sayısı: 747547
Farklı kelime sayısı: 161544
En çok tekrar eden 10 kelime: [('iklim', 25495), ('krizi', 21082), ('ve', 15160), ('bir', 9972), ('bu', 5269), ('için', 5057), ('#iklimkrizi', 5032), ('ile', 4429), ('de', 4066), ('da', 3734)]
En az tekrar eden 10 kelime: [('alınmamasına', 1), ('gireyim', 1), ('tutardı', 1), ('https://t.co/AgSr3bjsuO', 1), ('netliğiyle', 1), ('ilkeler.', 1), ('gazetecisi', 1), ('https://t.co/ABdKzTjvog', 1), ('sürdürülsün.', 1), ('santralleriyle', 1)]


Sütun 2: icerik_tr_en_char
Toplam kelime sayısı: 528373
Farklı kelime sayısı: 32760
En çok tekrar eden 10 kelime: [('iklim', 26677), ('krizi', 25839), ('bir', 10608), ('kuresel', 3722), ('klim', 3602), ('iklimkrizi', 3585), ('var', 2956), ('dunya', 2784), ('yok', 2496), ('degil', 2141)]

--------------------------------------------------------------------------

----

-------------------------------------------------------

## Twitter verileri 

In [274]:
veriler = pd.read_csv("twitter_dataset.csv")

In [276]:
veriler.head()

,Tweet_ID,Username,Text,Retweets,Likes,Timestamp
0,1,julie81,Party least receive say or single. Prevent pre...,2,25,2023-01-30 11:00:51
1,2,richardhester,Hotel still Congress may member staff. Media d...,35,29,2023-01-02 22:45:58
2,3,williamsjoseph,Nice be her debate industry that year. Film wh...,51,25,2023-01-18 11:25:19
3,4,danielsmary,Laugh explain situation career occur serious. ...,37,18,2023-04-10 22:06:29
4,5,carlwarren,Involve sense former often approach government...,27,80,2023-01-24 07:12:21


In [278]:
veriler.isna().sum()

Tweet_ID     0
Username     0
Text         0
Retweets     0
Likes        0
Timestamp    0
dtype: int64

In [280]:
metin_list = veriler["Text"].to_list()
metin_list_ham_veri = metin_list

In [282]:
metin_list[:200]

['Party least receive say or single. Prevent prevent husband affect. May himself cup style evening protect. Effect another themselves stage perform.\nPossible try tax share style television with. Successful much sell development economy effect.',
 'Hotel still Congress may member staff. Media draw buy fly. Identify on another turn minute would.\nLocal subject way believe which question some message. Own all imagine join agency indicate.',
 'Nice be her debate industry that year. Film where generation push discover partner level.\nNearly money store style may enjoy. Kid discuss blue save. Model another about along.\nEverybody especially dinner character yard.',
 'Laugh explain situation career occur serious. Five particular important size.\nCatch continue east teach dark discussion spring. Then candidate finally deep child. Big field billion force the there get.',
 'Involve sense former often approach government. While season family term close do number. Cost through second image indeed

    Adım adım ön işleme
- Lowercase
- Remove Punctuation
- Handling Hashtags, Mentions, URLs
- Remove Irrelevant Emojis
- Frequency-based Word Removal
- Remove Stopwords
- Lexicon Normalization (Lemmatization or Stemming)

In [285]:
## lowercase 
metin_list = [tweet.lower() for tweet in metin_list]
## noktalama işaretleri
metin_list = [''.join([char for char in tweet if char not in string.punctuation]) for tweet in metin_list]
## stopwords kelimeleri 
stop_words = set(stopwords.words('english'))  # veya Türkçe için 'turkish'
metin_list = [' '.join([word for word in tweet.split() if word not in stop_words]) for tweet in metin_list]
## mentions http olan içerikler
metin_list = [re.sub(r'@\w+|https?://\S+|#', '', tweet) for tweet in metin_list]
## emojiler
metin_list = [re.sub(r'[^\x00-\x7F]+', '', tweet) for tweet in metin_list]

print("*"*200)
print("Ham veri : ", metin_list_ham_veri[:10])
print("*"*200)
print("İşlenmiş veri : ",metin_list[:10])
print("*"*200)

********************************************************************************************************************************************************************************************************
Ham veri :  ['Party least receive say or single. Prevent prevent husband affect. May himself cup style evening protect. Effect another themselves stage perform.\nPossible try tax share style television with. Successful much sell development economy effect.', 'Hotel still Congress may member staff. Media draw buy fly. Identify on another turn minute would.\nLocal subject way believe which question some message. Own all imagine join agency indicate.', 'Nice be her debate industry that year. Film where generation push discover partner level.\nNearly money store style may enjoy. Kid discuss blue save. Model another about along.\nEverybody especially dinner character yard.', 'Laugh explain situation career occur serious. Five particular important size.\nCatch continue east teach dark discussio

In [287]:
## frekans
kelimeler = nltk.word_tokenize("".join(metin_list))
print("Tokens : ",kelimeler[:10])

Tokens :  ['party', 'least', 'receive', 'say', 'single', 'prevent', 'prevent', 'husband', 'affect', 'may']


In [289]:
frekans = Counter(kelimeler)
frekans

## kelime sıklıkları

Counter({'hard': 393,
         'tax': 382,
         'maybe': 377,
         'job': 376,
         'wear': 371,
         'instead': 370,
         'add': 370,
         'piece': 369,
         'check': 369,
         'young': 369,
         'level': 368,
         'teach': 368,
         'forget': 368,
         'success': 367,
         'high': 367,
         'soldier': 366,
         'food': 365,
         'voice': 364,
         'boy': 363,
         'majority': 363,
         'man': 363,
         'want': 362,
         'successful': 361,
         'agree': 361,
         'senior': 361,
         'yard': 361,
         'try': 360,
         'exactly': 360,
         'though': 360,
         'west': 359,
         'growth': 359,
         'able': 359,
         'rule': 359,
         'scene': 359,
         'team': 359,
         'dog': 359,
         'dark': 358,
         'room': 358,
         'account': 358,
         'cell': 357,
         'meeting': 357,
         'accept': 356,
         'edge': 356,
         'goal

### Lemmatization veya Stemming işlemi

In [292]:
# Kelimelerin POS etiketlerini al
pos_kelimeler = pos_tag(kelimeler)

# Lemmatizer'ı oluştur
lemmatizer = WordNetLemmatizer()

# Kelimeleri lemmatize et
lemma_kelimeler = [
    lemmatizer.lemmatize(kelime, pos=pos[0].lower()) if pos[0].lower() in ['a', 'r', 'n', 'v'] else lemmatizer.lemmatize(kelime)
    for kelime, pos in pos_kelimeler
]

In [293]:
print("Before : ", kelimeler[:90])
print("*" * 100)
print("After : ", lemma_kelimeler[:90])

Before :  ['party', 'least', 'receive', 'say', 'single', 'prevent', 'prevent', 'husband', 'affect', 'may', 'cup', 'style', 'evening', 'protect', 'effect', 'another', 'stage', 'perform', 'possible', 'try', 'tax', 'share', 'style', 'television', 'successful', 'much', 'sell', 'development', 'economy', 'effecthotel', 'still', 'congress', 'may', 'member', 'staff', 'media', 'draw', 'buy', 'fly', 'identify', 'another', 'turn', 'minute', 'would', 'local', 'subject', 'way', 'believe', 'question', 'message', 'imagine', 'join', 'agency', 'indicatenice', 'debate', 'industry', 'year', 'film', 'generation', 'push', 'discover', 'partner', 'level', 'nearly', 'money', 'store', 'style', 'may', 'enjoy', 'kid', 'discuss', 'blue', 'save', 'model', 'another', 'along', 'everybody', 'especially', 'dinner', 'character', 'yardlaugh', 'explain', 'situation', 'career', 'occur', 'serious', 'five', 'particular', 'important', 'size']
***********************************************************************************

In [296]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(metin_list)

In [306]:

# Çıktıyı yan yana görebilmek adına
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 250)
# Kelimeleri sütun isimleri olarak alıyoruz
df_original = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

print("Orijinal Kelime Matrisi (Var/Yok)")
print(df_original.iloc[:10,100:110])

# Lemmatization işlemi için WordNetLemmatizer kullanıyoruz
lemmatizer = WordNetLemmatizer()

# Her kelimenin türünü bulup lemmatize ediyoruz
words_pos = pos_tag(vectorizer.get_feature_names_out())
lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in words_pos]

# Lemmatized kelimelerle bir dict oluşturup aynı kelimeyi tekrar etmemesi için set kullanıyoruz
lemmatized_word_dict = {word: lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in words_pos}
df_lemmatized = df_original.rename(columns=lemmatized_word_dict)

# Aynı köke sahip olan kelimeleri birleştiriyoruz
df_lemmatized = df_lemmatized.groupby(axis=1, level=0, sort=False).sum()

print("\nLemmatization Sonrası Kelime Matrisi (Var/Yok)")
print(df_lemmatized.iloc[:10,100:110])

Orijinal Kelime Matrisi (Var/Yok)
   build  building  business  buy  call  camera  campaign  candidate  capital  car
0      0         0         0    0     0       0         0          0        0    0
1      0         0         0    1     0       0         0          0        0    0
2      0         0         0    0     0       0         0          0        0    0
3      0         0         0    0     0       0         0          1        0    0
4      0         0         0    0     0       0         0          0        0    0
5      0         0         0    0     0       0         0          0        0    0
6      1         0         0    0     0       0         0          0        0    0
7      0         0         0    0     0       0         0          0        0    0
8      0         0         0    0     0       0         0          0        0    0
9      0         0         0    0     0       0         0          0        0    0

Lemmatization Sonrası Kelime Matrisi (Var/Yok)
   bu

In [318]:
veriler.head()

,Tweet_ID,Username,Text,Retweets,Likes,Timestamp,islenmis_icerik
0,1,julie81,Party least receive say or single. Prevent pre...,2,25,2023-01-30 11:00:51,party least receive say single prevent prevent...
1,2,richardhester,Hotel still Congress may member staff. Media d...,35,29,2023-01-02 22:45:58,hotel still congress may member staff media dr...
2,3,williamsjoseph,Nice be her debate industry that year. Film wh...,51,25,2023-01-18 11:25:19,nice debate industry year film generation push...
3,4,danielsmary,Laugh explain situation career occur serious. ...,37,18,2023-04-10 22:06:29,laugh explain situation career occur serious f...
4,5,carlwarren,Involve sense former often approach government...,27,80,2023-01-24 07:12:21,involve sense former often approach government...


In [320]:
metin_list[:]

['party least receive say single prevent prevent husband affect may cup style evening protect effect another stage perform possible try tax share style television successful much sell development economy effect',
 'hotel still congress may member staff media draw buy fly identify another turn minute would local subject way believe question message imagine join agency indicate',
 'nice debate industry year film generation push discover partner level nearly money store style may enjoy kid discuss blue save model another along everybody especially dinner character yard',
 'laugh explain situation career occur serious five particular important size catch continue east teach dark discussion spring candidate finally deep child big field billion force get',
 'involve sense former often approach government season family term close number cost second image indeed production thousand natural research land bank option one party nation mother appear leader student discussion region arrive idea ano

In [328]:
print(type(frekans))
print(frekans)

<class 'collections.Counter'>
Counter({'hard': 393, 'tax': 382, 'maybe': 377, 'job': 376, 'wear': 371, 'instead': 370, 'add': 370, 'piece': 369, 'check': 369, 'young': 369, 'level': 368, 'teach': 368, 'forget': 368, 'success': 367, 'high': 367, 'soldier': 366, 'food': 365, 'voice': 364, 'boy': 363, 'majority': 363, 'man': 363, 'want': 362, 'successful': 361, 'agree': 361, 'senior': 361, 'yard': 361, 'try': 360, 'exactly': 360, 'though': 360, 'west': 359, 'growth': 359, 'able': 359, 'rule': 359, 'scene': 359, 'team': 359, 'dog': 359, 'dark': 358, 'room': 358, 'account': 358, 'cell': 357, 'meeting': 357, 'accept': 356, 'edge': 356, 'goal': 356, 'number': 355, 'work': 355, 'within': 355, 'term': 354, 'since': 354, 'serve': 354, 'site': 353, 'personal': 353, 'anyone': 353, 'science': 353, 'age': 353, 'son': 353, 'owner': 353, 'entire': 353, 'paper': 353, 'light': 353, 'relationship': 353, 'receive': 352, 'send': 352, 'speak': 352, 'deal': 352, 'dream': 352, 'long': 352, 'tend': 352, 'gener

In [338]:
least = frekans.values()
least_key = frekans.keys()

In [342]:
print(least)
print(least_key)

dict_values([327, 328, 352, 314, 338, 320, 350, 337, 324, 331, 315, 326, 335, 327, 305, 330, 323, 327, 360, 382, 320, 326, 361, 321, 320, 318, 300, 1, 320, 314, 327, 342, 305, 313, 312, 336, 307, 336, 333, 342, 321, 317, 323, 341, 335, 342, 333, 333, 335, 1, 333, 319, 315, 302, 351, 320, 313, 329, 368, 346, 328, 316, 347, 344, 285, 300, 325, 320, 344, 307, 316, 314, 314, 1, 305, 342, 345, 291, 309, 312, 337, 335, 310, 297, 311, 331, 368, 358, 324, 307, 321, 336, 316, 333, 319, 326, 315, 347, 1, 312, 328, 313, 342, 286, 311, 348, 354, 340, 355, 292, 330, 351, 303, 341, 317, 318, 325, 325, 309, 327, 346, 322, 320, 345, 318, 345, 309, 336, 310, 1, 324, 334, 319, 332, 339, 318, 336, 341, 334, 350, 327, 331, 325, 323, 325, 318, 323, 278, 328, 326, 327, 317, 376, 311, 328, 299, 324, 330, 333, 1, 315, 292, 298, 303, 321, 322, 321, 322, 340, 318, 357, 353, 322, 340, 336, 340, 341, 278, 284, 333, 309, 288, 303, 309, 314, 340, 324, 1, 330, 332, 342, 323, 350, 333, 304, 336, 345, 369, 310, 312, 3

In [358]:
def least_words(counter, n=10):
    # En az tekrar eden n kelimeyi al
    least = counter.most_common()[:-n-1:-1]  # En az tekrar eden n kelime
    least_keys, least_values = zip(*least)  # Anahtar ve değerleri ayır
    a = list(zip(least_keys, least_values))  # Anahtar ve değerleri birleştir ve listeye dönüştür
    # Sonuçları döndür
    return a

def most_words(counter,n=10):
    most = counter.most_common(10)
    least_keys, least_values = zip(*most)  # Anahtar ve değerleri ayır
    a = list(zip(least_keys, least_values))  # Anahtar ve değerleri birleştir ve listeye dönüştür
    return a


print("En az tekrar eden kelimeler ve sıklıkları : ",least_words(frekans))
print("*"*100)
print("En fazla tekrar eden kelimeler ve sıklıkları : ",most_words(frekans))

En az tekrar eden kelimeler ve sıklıkları :  [('neverbody', 1), ('lineguess', 1), ('kindday', 1), ('seemborn', 1), ('manyagree', 1), ('becomeboy', 1), ('safemajor', 1), ('militarylisten', 1), ('troubledoctor', 1), ('viewrepresent', 1)]
****************************************************************************************************
En fazla tekrar eden kelimeler ve sıklıkları :  [('hard', 393), ('tax', 382), ('maybe', 377), ('job', 376), ('wear', 371), ('instead', 370), ('add', 370), ('piece', 369), ('check', 369), ('young', 369)]


-----------------------------

------

-------

## Gender Classifier

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn

In [28]:
data = pd.read_csv("gender_classifier.csv",encoding="latin1")
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [30]:
data.shape

(20050, 26)

In [150]:
data1 = pd.concat([data.gender,data.description],axis=1)
data1.head(10)

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...
5,female,you don't know me.
6,brand,"A global marketplace for images, videos and mu..."
7,male,The secret of getting ahead is getting started.
8,female,Pll Fan // Crazy about MCD // Ramen is bae
9,female,"Renaissance art historian, University of Notti..."


In [152]:
data1.shape

(20050, 2)

In [154]:
data1.dropna(inplace=True)
data1.reset_index(drop=True,inplace=True)
print(data1.shape)

(16224, 2)


In [156]:
data1.gender = [1 if i == "female" else 0 for i in data1.gender]
data1.head()

,gender,description
0,0,i sing my own rhythm.
1,0,I'm the author of novels filled with family dr...
2,0,louis whining and squealing and all
3,0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,1,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [158]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [160]:
lemma = WordNetLemmatizer()

In [162]:
liste = []

for i in range(1000):
    metin = re.sub("[^a-zA-Z]"," ",data1.description[i])
    metin = metin.lower()
    metin = metin.split()
    metin = [lemma.lemmatize(j) for j in metin if not j in set(stopwords.words("english"))]
    metinson = " ".join(metin)
    liste.append(metinson)

In [163]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000)
x = cv.fit_transform(liste).toarray()
y = data1.iloc[:1000,0].values

In [164]:
liste[:10]

['sing rhythm',
 'author novel filled family drama romance',
 'louis whining squealing',
 'mobile guy er shazam google kleiner perkins yahoo sprint pc airtouch air force stanford gsb uva dad husband brother golfer',
 'ricky wilson best frontman kaiser chief best band xxxx thank kaiser chief incredible year gig memory cherish always xxxxxxx',
 'know',
 'global marketplace image video music sharing photo inspiration design tip video creative community',
 'secret getting ahead getting started',
 'pll fan crazy mcd ramen bae',
 'renaissance art historian university nottingham fuelled haribo partial coffee soft spot renaissance china national teaching fellow']

In [168]:
from sklearn.model_selection import train_test_split

In [170]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [172]:
from sklearn.naive_bayes import GaussianNB
gn = GaussianNB()
gn.fit(x_train,y_train)
yhead = gn.predict(x_test)

In [174]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,yhead)
print(cm)

[[96 46]
 [32 26]]


In [176]:
from sklearn.metrics import accuracy_score
print(accuracy_score(yhead, y_test))  # Compare predictions with true labels

0.61


In [180]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore") 

In [186]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
models = {
    'Logistic Regression': LogisticRegression(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier()
}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model_name)
    print(f"Accuracy score : {accuracy_score(y_pred, y_test)}")
    print("\n" + "-"*40 + "\n")


Logistic Regression
Accuracy score : 0.6933333333333334

----------------------------------------

XGBoost
Accuracy score : 0.6733333333333333

----------------------------------------

K-Nearest Neighbors
Accuracy score : 0.54

----------------------------------------

Random Forest
Accuracy score : 0.6933333333333334

----------------------------------------

